In [ ]:
# Show Result on the right hand side
from IPython.core.display import HTML
display(HTML("""
        <style>
        #notebook-container {
            width: 100%
        }

        .code_cell {
           flex-direction: row !important;
        }

        .code_cell .input {
            width: 50%
        }

        .code_cell .output_wrapper {
            width: 50%
        }
        </style>
        """))


In [ ]:
!nvidia-smi

In [ ]:
from IPython.display import clear_output

TEST
https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

# CREAT MODELS

First : Decide What hardware to use to train model.

In [ ]:
# Store devices into variable device
import torch
from torch import nn
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten  = nn.Flatten()
        self.linear_1 = nn.Linear(28*28,512)
        self.relu_1   = nn.ReLU()
        self.linear_2 = nn.Linear(512,1000)
        self.relu_2   = nn.ReLU()
        self.linear_3 = nn.Linear(1000,10)
    
    def forward(self,x):
        x = self.flatten(x)
        x = self.linear_1(x)
        x = self.relu_1(x)
        x = self.linear_2(x)
        x = self.relu_2(x)
        x = self.linear_3(x)
        return x

In [ ]:
# https://www.kaggle.com/code/pankajj/fashion-mnist-with-pytorch-93-accuracy/notebook
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

In [ ]:
# model = NeuralNetwork() # for cpu
model = NeuralNetwork().to(device) # for gpu
print(model)

# Create Loss and Optim Func

In [ ]:
# We need loss_func
loss_func = nn.CrossEntropyLoss()
# We need optimizer
opt = torch.optim.SGD(model.parameters(),lr=1e-3)

# Load Data

In [ ]:
# Create dataloader dataset and transform to tensor
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
# Load data in
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=batch_size,shuffle=True)

# View Plot data Leng

In [ ]:
# View Shape and plot leng
import matplotlib.pyplot as plt

for X, y in test_dataloader:
    print(X.shape)
    print(X[0].shape)
    print(torch.squeeze(X[0]).shape)
    plt.imshow(torch.squeeze(X[0]))
    print(y.shape)
    print(y[0])
    break

# Create Training FUnc

In [ ]:
# Let create train func
def train(dataloader, model, loss_func, opt):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)
        
        # 1st prediction compt
        pred = model(X)
        
        # 2nd loss compt
        loss = loss_func(pred,y)
        
        # 3rd Back prog
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Create Test FUnc

In [ ]:
# Let Create test func
def test(dataloader, model, loss_func):
    size = len(dataloader.dataset)
    num_bat = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_func(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_bat
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Start TRaining

In [ ]:
# Let start training and testing
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_func, opt)
    test(test_dataloader, model, loss_func)
print("Done!")

# SAVE AND LOAD MODEL

In [ ]:
# save model
import torch
torch.save(model.state_dict(), "./save_model/fashion_model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
# Load model
model = NeuralNetwork()
model.load_state_dict(torch.load("./save_model/fashion_model.pth"))

# Start Prediction

In [ ]:
classes_list = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"]

model.eval()
pic = 70
x = test_data[pic][0]
y = test_data[pic][1]
with torch.no_grad():
    pred      = model(x)
    predicted = classes_list[pred[0].argmax(0)]
    actual    = classes_list[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')